In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
def Labels(_df):
    def one_hot(x):
        '''
          对10个类型进行one_hot编码
        '''
        returnList=np.zeros([10])
        returnList[x]=1
        return(returnList)
    labels=_df.icol[0]  #取第一列
    temp=list()
    for i in labels:
        temp=temp.append(one_hot(i))
    returnMatrix=np.matrix(temp)
    return(returnMatrix)   #预计返回42000*10的矩阵
def train_test_data(_df):
    df=_df.copy()
    del df['label'] #删除label列
    returnMatrix=np.matrix(df)
    return(returnMatrix)   #预计返回42000*784的稀疏矩阵
def train_test_label(_df):
    '''
       返回训练数据集的label
    '''
    return(Labels(_df))
def random_batch(batch_size=100,train_X,train_Y):
    '''
       随机选择样本进行训练
    '''
    index=np.random.randint(0,train_X.shape[0]-batch_size)
    return(train_X[index:batch_size],train_Y[index:batch_size])
df=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")
dim=df.shape[0]
train_X=train_test_data(df)  #训练矩阵（每一个样本占一行）
test_X=train_test_data(df_test)
train_Y=train_test_label(df)  #训练集labels
test_Y=train_test_label(df_test)
shape=train_X.shape
sess=tf.InteractiveSession()
#创建真实label的输入值
y_=tf.placeholder(dtype=tf.float32,shape=[dim])
#创建训练矩阵输入
x=tf.placeholder(dtype=tf.float32,shape=shape)
#初始化权值
weights=tf.Variable(tf.zeros([shape[1],10]))
#初始化偏置值
bias=tf.Variable(tf.zeros([10]))
#实现softmax算法
y=tf.nn.softmax(tf.matmul(x,weights)+bias)
#定义损失函数loss，采用cross_entropy
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
#定义优化损失函数的优化器,这里采用SGD
train_step=tf.train.GradientDescentOptimizer(cross_entropy)
#定义计算预测精度函数
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
#进入训练阶段
#运行初始化计算子图
tf.global_variables_initializer().run()

for i in range(1000):
    batch_X,batch_Y=random_batch(batch_size=100)
    train_step.run(feed_dict={x:batch_X,y_:batch_Y})
    if i%100==0:
        print(accuracy.eval(feed_dict={x:test_X,y_:test_Y}))
print("accuracy:%f"%(accuracy.eval(feed_dict={x:test_X,y_:test_Y})))        
